<a href="https://colab.research.google.com/github/SChoi005/DeepLearningBasics/blob/main/data_validation_with_tfdv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Validation

In [3]:
!apt-get install python-dev python-snappy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  python-snappy
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 10.8 kB of archives.
After this operation, 39.9 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-snappy amd64 0.5-1.1build2 [10.8 kB]
Fetched 10.8 kB in 0s (56.7 kB/s)
Selecting previously unselected package python-snappy.
(Reading database ... 155639 files and directories currently installed.)
Preparing to unpack .../python-snappy_0.5-1.1build2_amd64.deb ...
Unpacking python-snappy (0.5-1.1build2) ...
Setting up python-snappy (0.5-1.1build2) ...


In [4]:
!pip install -q tensorflow_data_validation

     |████████████████████████████████| 10.9 MB 3.2 MB/s 
     |████████████████████████████████| 51 kB 7.3 MB/s 
     |████████████████████████████████| 99 kB 6.8 MB/s 
     |████████████████████████████████| 19.2 MB 315 kB/s 
     |████████████████████████████████| 23.6 MB 87.1 MB/s 
     |████████████████████████████████| 294 kB 72.4 MB/s 
     |████████████████████████████████| 511.7 MB 5.0 kB/s 
     |████████████████████████████████| 272 kB 54.5 MB/s 
     |████████████████████████████████| 46 kB 3.4 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 2.3 MB 41.6 MB/s 
     |████████████████████████████████| 508 kB 48.9 MB/s 
     |████████████████████████████████| 151 kB 60.7 MB/s 
     |████████████████████████████████| 435 kB 53.0 MB/s 
     |████████████████████████████████| 118 kB 57.9 MB/s 
     |████████████████████████████████| 234 kB 52.3 MB/s 
     |████████████████████████████████| 255 kB 60.9 MB/s 
     |██████████████

# Import project dependencies

In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
from __future__ import print_function

## Simple dataset analysis

In [4]:
dataset = pd.read_csv("pollution_small.csv")

In [5]:
dataset.shape

(2188, 5)

In [6]:
training_data = dataset[:1600]

In [7]:
training_data.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


In [8]:
test_set = dataset[1600:]

In [9]:
test_set.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


## Data analysis and validation with TFDV

### Generate traing data statistics

In [10]:
train_stats = tfdv.generate_statistics_from_dataframe(dataframe=dataset)

### Infering the schema

In [11]:
schema = tfdv.infer_schema(statistics=train_stats)

In [12]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'so2',FLOAT,required,,-
'soot',FLOAT,required,,-


### Calculate test set statistics

In [13]:
test_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set)

## Campare test statistics with the Schema

### Checking for anomalies in new data

In [14]:
anomalies = tfdv.validate_statistics(statistics=test_stats, schema=schema)

### Displaying all detected anomalies
* Integer larger than 10
* STRING type when expected INT type
* FLOAT type when expected INT type
* Integer smaller than 0

In [15]:
tfdv.display_anomalies(anomalies)

### New data WITH anomalies

In [16]:
test_set_copy = test_set.copy()

In [17]:
test_set_copy.drop("soot", axis=1, inplace=True)

### Statistics based on data with anomalies

In [18]:
test_set_copy_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set_copy)

In [19]:
anomalies_new = tfdv.validate_statistics(statistics=test_set_copy_stats, schema=schema)

In [20]:
tfdv.display_anomalies(anomalies_new)

,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


## Prepare the schema for Serving

In [21]:
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

### Removing a target column from the Serving schema

In [22]:
tfdv.get_feature(schema, "soot").not_in_environment.append("SERVING")

### Checking for anomalies between theSERVING environment and new test set

In [23]:
serving_env_anomalies = tfdv.validate_statistics(test_set_copy_stats, schema, environment="SERVING")

In [24]:
tfdv.display_anomalies(serving_env_anomalies)

## Freezing the schema

In [25]:
tfdv.write_schema_text(schema=schema, output_path="pollution_schema.pbtxt")